In [1]:
from jettool import tools
import pandas
import numpy

In [2]:
# 初始化物件，可同時set Key
tejtool = tools.financial_tool(api_key="GDEy0mWAGqnI3EemCREGREZMcEVbnF")
# 設定keyID，會自動查詢需要的資料表索引構造
#tejtool.set_apikey("GDEy0mWAGqnI3EemCREGREZMcEVbnF")

['使用者名稱：趙育祥(ZYX)', '使用權限日期：2018-01-01/2050-12-31', '日連線次數狀態：369/100000', '日查詢資料量狀態：1105991/10000000', '月查 詢資料量狀態：12011873/9223372036854775807']


In [ ]:
#尋找某個table的mapping資料次屬性
tejtool.get_table_mapping(market='TWN',id='AIND')

In [ ]:
# 用模糊字串來查詢確切的會計科目名稱
tejtool.find_account_name('資產負債')

### 一次查詢全部整合完成
#### 一個名稱、可到不同資料表查詢
#### 需正確定義名稱的精確差異

In [3]:
#left_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','個別風險','長期反轉(T-60~T-13)','董監持股%','大股東持股(TSE)%','經理人持股%','單月營收(千元)']
left_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','單月營收(千元)']
data = tejtool.query_data(base_date='2019-12-31',window='2m',column_names=left_name)
#tejtool.save_data()
data[data['coid']=='2330']

資料起始日：2019-10-31
查詢財報資料
成功查詢會計家數:1625
最大財報資料日期:2019-12-01
查詢日資料 最大資料日期:2019-12-31
借券餘額 重新查詢:True
上市(櫃)調整股價(日)-除權息調整 重新查詢:True
上市(櫃)月營收盈餘 重新查詢:True


,zdate,mdate,coid,收盤價(元),單月營收(千元),常續性稅後淨利,報酬率-Ln
11063,2019-12-31,2019-09-01,2330,331.0,107884396.0,0.0,-1.0519
11064,2019-12-30,2019-09-01,2330,334.5,107884396.0,0.0,-1.0409
11065,2019-12-27,2019-09-01,2330,338.0,107884396.0,0.0,1.4903
11066,2019-12-26,2019-09-01,2330,333.0,107884396.0,0.0,0.0000
11067,2019-12-25,2019-09-01,2330,333.0,107884396.0,0.0,0.3008
11068,2019-12-24,2019-09-01,2330,332.0,107884396.0,0.0,-0.6006
11069,2019-12-23,2019-09-01,2330,334.0,107884396.0,0.0,1.5083
11070,2019-12-20,2019-09-01,2330,329.0,107884396.0,0.0,-1.8073
11071,2019-12-19,2019-09-01,2330,335.0,107884396.0,0.0,-2.0529
11072,2019-12-18,2019-09-01,2330,344.5,107884396.0,0.0,-0.1450


In [ ]:
def calculate():
    tejtool.data['EP'] = tejtool.data['常續性稅後淨利']/tejtool.data['收盤價(元)']
    tejtool.data['常續性稅後淨利檢查'] = tejtool.check_above('EP',0,window='1d')
def evaluate():
    tejtool.data['購入'] = tejtool.check_condition(conditions=['常續性稅後淨利檢查'])
    tejtool.data['unit'] = tejtool.equal_pv()
    
tejtool.back_test(back_interval='2019-10-31',cash=1000000000,calculate=calculate,evaluate=evaluate)